# Prompt Parallelization Workflow
In this notebook, we will build a content moderation system specifically designed for media and entertainment companies using the Prompt Parallelization pattern with Stands Agent. This pattern allows us to process media content through multiple specialized LLM prompts in parallel and then aggregate the results.

At a high level, we'll be creating a media content moderation system that:

1. Takes content as input (scripts, user comments, social posts, etc.)
2. Analyzes it from multiple perspectives in parallel:
  - Explicit language detection and age-appropriate terminology
  - Inappropriate themes evaluation and cultural sensitivity
  - Misleading content identification and copyright concerns
  - Sentiment analysis and brand alignment assessment
3. Aggregates the results into a comprehensive moderation report tailored for media platforms


The following diagram depicts the workflow for a parallelization workflow implemented in Strands Agent:

<img src="../../imgs/strands-parallel-llm-architecture.png" width="800"> 


## Package Installation
This cell installs the required Strands Agent packages needed for this workflow. The `-Uq` flags ensure packages are upgraded to the latest version (`-U`) and installed quietly (`-q`) without verbose output.

**Purpose**: Sets up the necessary dependencies for building and running the parallel content moderation system.


In [ ]:
%pip install strands-agents strands-agents-tools -Uq

## Environment Setup

### Loading Environment Variables
setting up logger, initiates strands Agent libraries

In [1]:
from strands import Agent
import asyncio

## Model Configuration
This cell defines the AWS Bedrock model to be used across all agents in the workflow. In our example, we'll use `us.amazon.nova-pro-v1:0`.

In [28]:
model_id = "us.amazon.nova-pro-v1:0"

## Explicit Language Analysis Agent
In the following, we create a specialized agent focused on analyzing content for explicit language and age-appropriate terminology. The agent is configured with a detailed system prompt that instructs it to:

1. **Identify explicit language and profanity**
2. **Assess age-appropriate terminology**
3. **Recommend content ratings (G, PG, PG-13, R, etc.)**
4. **Provide revision recommendations**

The agent returns structured JSON output with assessment, instances of inappropriate content, age rating, and revision recommendations.

In [29]:
# Create specialized agents
explicitLanguageAnalysisAgent_system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying explicit language and age-appropriate content.

# Analyze the following content for:

1. Explicit language and profanity
2. Age-appropriate terminology
3. Content suitable for different audience age ratings (G, PG, PG-13, R, etc.)
4. Dialogue or descriptions that may require parental guidance

## Provide a detailed report with:

1. Overall assessment (Family-Friendly/Teen-Appropriate/Mature/Adult-Only)
2. Specific instances of explicit language or age-inappropriate content found (if any)
3. Age rating recommendation (G, PG, PG-13, R, NC-17, etc.)
4. Recommendations for content revision to meet specific audience age targets

Format your response as JSON:

   {
      "assessment": "Family-Friendly/Teen-Appropriate/Mature/Adult-Only",
      "instances": ["example1", "example2"],
      "age_rating": "PG-13",
      "revision_recommendations": "Your recommendations here"
   }
"""
explicitLanguageAnalysisAgent = Agent(model=model_id, system_prompt=explicitLanguageAnalysisAgent_system_prompt, callback_handler=None)


## Inappropriate Themes Analysis Agent
In the followiong, we create the second specialized agent that focuses on identifying inappropriate themes and cultural sensitivity issues. The agent analyzes content for:

1. **Inappropriate themes** (violence, discrimination, harmful ideologies)
2. **Cultural sensitivity concerns** across global audiences
3. **Offensive stereotypes or representations**
4. **Regional market appropriateness**

The agent provides a structured JSON response with overall assessment, identified themes, sensitive regions, and revision recommendations.

In [ ]:
inappropriateThemesAnalysisAgent_system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying inappropriate themes and cultural sensitivity issues.

# Analyze the following content for:

1. Inappropriate themes such as violence, discrimination, or harmful ideologies
2. Cultural sensitivity concerns across global audiences
3. Potentially offensive stereotypes or representations
4. Content that may be inappropriate for certain regions or markets


## Provide a detailed report with:

1. Overall assessment (Universally Acceptable/Regionally Sensitive/Potentially Offensive)
2. Specific inappropriate themes or cultural sensitivity issues identified (if any)
3. Regions or demographics that may find the content problematic
4. Recommendations for content revision to improve cultural sensitivity

# Format your response as JSON:

   {
      "assessment": "Universally Acceptable/Regionally Sensitive/Potentially Offensive",
      "themes": ["theme1", "theme2"],
      "sensitive_regions": ["region1", "region2"],
      "revision_recommendations": "Your recommendations here"
   }
"""
inappropriateThemesAnalysisAgent = Agent(model=model_id, system_prompt=inappropriateThemesAnalysisAgent_system_prompt, callback_handler=None)


## Misleading Content Analysis Agent
The following cell creates a specialized agent responsible for identifying misleading content, factual inaccuracies, and potential copyright issues. The agent examines:

1. **Misleading claims or factual inaccuracies**
2. **Potential copyright or intellectual property concerns**
3. **Unauthorized references** to protected brands, characters, or properties
4. **Content requiring fact-checking or legal review**

The agent returns structured JSON with factual assessment, identified inaccuracies, copyright concerns, and revision recommendations.

In [31]:
misleadingContentAnalysis_system_prompt = """You are a content moderator for a major media and entertainment company specializing in identifying misleading content, factual inaccuracies, and potential copyright issues.

# Analyze the following content for:

1. Misleading claims or factual inaccuracies
2. Potential copyright or intellectual property concerns
3. Unauthorized references to protected brands, characters, or properties
4. Content that may require fact-checking or legal review

## Provide a detailed report with:

1. Overall assessment (Factually Sound/Needs Verification/Potentially Infringing)
2. Specific misleading claims or factual inaccuracies identified (if any)
3. Potential copyright or IP concerns
4. Recommendations for content revision to address factual or legal issues

Format your response as JSON:

   {
      "assessment": "Factually Sound/Needs Verification/Potentially Infringing",
      "inaccuracies": ["inaccuracy1", "inaccuracy2"],
      "copyright_concerns": ["concern1", "concern2"],
      "revision_recommendations": "Your recommendations here"
   }
"""
misleadingContentAnalysisAgent = Agent(model=model_id, system_prompt=misleadingContentAnalysis_system_prompt, callback_handler=None)

## Sentiment & Brand Alignment Analysis Agent
The following cell creates our fourth specialized agent that analyzes content for sentiment and brand alignment. The agent evaluates:

1. **Overall emotional tone and sentiment**
2. **Alignment with media brand values** (positivity, inclusivity, creativity)
3. **Potential impact on audience perception and brand reputation**
4. **Emotional resonance with target demographics**

The agent provides structured JSON output with sentiment assessment, brand alignment evaluation, emotional impact analysis, and revision recommendations.

In [32]:
sentimentBrandAnalysis_system_prompt = """You are a content moderator for a major media and entertainment company specializing in sentiment analysis and brand alignment.

# Analyze the following content for:

1. Overall emotional tone and sentiment
2. Alignment with typical media brand values (positivity, inclusivity, creativity, etc.)
3. Potential impact on audience perception and brand reputation
4. Emotional resonance with target demographics

# Provide a detailed report with:
1. Overall sentiment (Positive/Neutral/Negative)
2. Brand alignment assessment (Aligned/Partially Aligned/Misaligned)
3. Potential emotional impact on viewers/readers
4. Recommendations for content revision to improve brand alignment

Format your response as JSON:

{
    "sentiment": "Positive/Neutral/Negative",
    "brand_alignment": "Aligned/Partially Aligned/Misaligned",
    "emotional_impact": "Your assessment here",
    "revision_recommendations": "Your recommendations here"
}
"""
sentimentBrandAnalysisAgent = Agent(model=model_id, system_prompt=sentimentBrandAnalysis_system_prompt, callback_handler=None)


## Media Platform Recommendations Agent
This following cell creates our  aggregation agent that synthesizes all previous analyses into comprehensive platform-specific recommendations. This senior moderator agent:

1. **Aggregates results** from all four specialized agents
2. **Makes overall moderation decisions** (Approve/Revise/Reject)
3. **Provides platform-specific recommendations** for different distribution channels
4. **Suggests content tags and warnings**
5. **Justifies decisions** with comprehensive reasoning

The agent uses a template to combine all previous reports and generates actionable recommendations for streaming platforms, broadcast TV, social media, and international distribution.

In [38]:
mediaPlatformRecommendations_system_prompt = """You are a senior content moderator for a major media and entertainment company responsible for creating comprehensive moderation reports with platform-specific recommendations.

   You are given the moderation analysis reports as described below:
   
   - Explicit Language & Age-Appropriate Analysis
   - Inappropriate Themes & Cultural Sensitivity Analysis
   - Misleading Content & Copyright Analysis
   - Sentiment & Brand Alignment Analysis


   Based on these reports, provide:
   1. Overall moderation decision (Approve/Revise/Reject)
   2. Summary of key concerns from all analyses
   3. Platform-specific recommendations for:
      - Streaming platforms (Netflix, Disney+, Prime Video, etc.)
      - Broadcast television
      - Social media platforms
      - International distribution
   4. Specific content tags or warnings recommended
   5. Justification for your decision

   Format your response as a well-structured report with clear sections for each platform type.
"""

mediaPlatformRecommendations_prompt_template = """## Explicit Language & Age-Appropriate Analysis Report:

{explicitLanguageAnalysisAgent_report}

## Inappropriate Themes & Cultural Sensitivity Analysis Report:

{inappropriateThemesAnalysisAgent_report}

## Misleading Content & Copyright Analysis Report:

{misleadingContentAnalysisAgent_report}


## Sentiment & Brand Alignment Analysis Report:

{sentimentBrandAnalysisAgent_report}

"""
mediaPlatformRecommendationsAgent = Agent(model=model_id, system_prompt=mediaPlatformRecommendations_system_prompt, callback_handler=None)

## Asynchronous Agent Invocation Helper
This cell defines a simple helper function that wraps agent calls for asynchronous execution. The function takes an agent instance and a query, then returns the agent's response.

**Purpose**: Enables parallel execution of multiple agents using Python's `asyncio.gather()` function, which is essential for the prompt parallelization pattern.

In [34]:
async def invoke_agent_async(agent, query):
    return agent(query)

## Main Parallel Workflow Function
The following cell implements the core parallel processing workflow that orchestrates the entire content moderation system:

### **Step 1: Parallel Analysis**
- Executes all four specialized agents simultaneously using `asyncio.gather()`
- Processes the same content through multiple analytical lenses in parallel
- Significantly reduces total processing time compared to sequential execution

### **Step 2: Result Aggregation**
- Extracts analysis results from each agent's response
- Combines individual reports into a structured template
- Prepares comprehensive input for the final recommendation agent

### **Step 3: Final Recommendations**
- Passes aggregated reports to the media platform recommendations agent
- Generates platform-specific guidance and moderation decisions

**Purpose**: Implements the prompt parallelization pattern to efficiently process content through multiple specialized AI agents and produce comprehensive moderation reports.

In [43]:
# Sequential workflow processing
async def process_parallel_workflow(query):
    
    # Step 1: Explicit Language Analysis Agent
    prompt = f"Content to analyze: {query}"
    parallel_call_results = await asyncio.gather(invoke_agent_async(agent=explicitLanguageAnalysisAgent, query=prompt),
                                                invoke_agent_async(agent=inappropriateThemesAnalysisAgent, query=prompt),
                                                invoke_agent_async(agent=misleadingContentAnalysisAgent, query=prompt),
                                                invoke_agent_async(agent=sentimentBrandAnalysisAgent, query=prompt))
    
    explicitLanguageAnalysisAgent_report = parallel_call_results[0].message['content'][0]['text']
    inappropriateThemesAnalysisAgent_report = parallel_call_results[1].message['content'][0]['text']
    misleadingContentAnalysisAgent_report = parallel_call_results[2].message['content'][0]['text']
    sentimentBrandAnalysisAgent_report = parallel_call_results[3].message['content'][0]['text']

    mediaPlatformRecommendations_prompt = mediaPlatformRecommendations_prompt_template.replace("{explicitLanguageAnalysisAgent_report}", explicitLanguageAnalysisAgent_report)
    mediaPlatformRecommendations_prompt = mediaPlatformRecommendations_prompt.replace("{inappropriateThemesAnalysisAgent_report}", inappropriateThemesAnalysisAgent_report)
    mediaPlatformRecommendations_prompt = mediaPlatformRecommendations_prompt.replace("{misleadingContentAnalysisAgent_report}", misleadingContentAnalysisAgent_report)
    mediaPlatformRecommendations_prompt = mediaPlatformRecommendations_prompt.replace("{sentimentBrandAnalysisAgent_report}", sentimentBrandAnalysisAgent_report)

    mediaPlatformRecommendations_result = mediaPlatformRecommendationsAgent(mediaPlatformRecommendations_prompt)
    return mediaPlatformRecommendations_result

## Testing
We are going to test the workflow using a sample script excerpt that will be used to demonstrate the content moderation workflow. 

The script is a crime drama scene that contains:

- **Mature language** (profanity, strong expressions)
- **Violence themes** (murder investigation, crime scene)
- **Dark tone** (noir-style detective story)
- **Potentially sensitive content** (smoking, explicit references)

This sample represents typical content that would require thorough moderation analysis before distribution across different media platforms.

In [44]:
script = """FADE IN:

   EXT. CITY STREET - NIGHT

   Rain pours down as DETECTIVE MORGAN (45, hardened) stands over a body covered by a sheet. Lightning flashes, illuminating the grim scene.

   DETECTIVE MORGAN
   (lighting a cigarette)
   Third one this month. Same damn signature.

   OFFICER JENKINS (30s) approaches, looking queasy.

   OFFICER JENKINS
   The mayor's going to have a fit. The press is already calling this guy "The Midnight Butcher."

   DETECTIVE MORGAN
   I don't give a damn what they call him. I'm going to catch this son of a bitch before he cuts up another girl.

   Morgan kneels down and pulls back the sheet, revealing a young woman's face, pale and lifeless.

   DETECTIVE MORGAN
   (whispering)
   I promise you justice, sweetheart. Whatever it takes.

   He stands, tosses his cigarette into a puddle.

   DETECTIVE MORGAN
   Jenkins, get me everything on the victim. And I mean everything. Where she worked, who she dated, what goddamn toothpaste she used.

   OFFICER JENKINS
   But sir, it's almost midnight and—

   DETECTIVE MORGAN
   (interrupting)
   Evil doesn't sleep, why should we?

   Morgan walks away as lightning flashes again, his silhouette momentarily frozen against the night sky.

   FADE OUT.
   """

In [ ]:
response = await process_parallel_workflow(query=script)

## Final Results
Examine the moderation report generated by the parallel workflow. 

In [ ]:
print(response)

## Summary
This notebook demonstrated the Prompt Parallelization pattern using Strands Agent to build a comprehensive content moderation system for media and entertainment companies. The system analyzes content simultaneously from multiple specialized perspectives and aggregates results into actionable platform-specific recommendations.

### Key Components Demonstrated:
1. Specialized Analysis Agents:
Four specialized agents were created to analyze content from different perspectives, and a recommendation agent that synthesize the results from all sub agents.

- Explicit Language Analysis: Identifies profanity, age-inappropriate content, and provides content rating recommendations (G, PG, PG-13, R, etc.)
- Inappropriate Themes Analysis: Evaluates violence, discrimination, cultural sensitivity, and regional appropriateness
- Misleading Content Analysis: Detects factual inaccuracies, copyright concerns, and intellectual property issues
- Sentiment & Brand Alignment: Analyzes emotional tone, brand value alignment, and audience perception impact
- Media Platform Recommendations Agent: Synthesizes all four analyses into comprehensive platform-specific guidance

2. Parallel Processing Architecture
- Execute all four specialized agents simultaneously rather than sequentially, significantly reduced total processing time while maintaining comprehensive analysis depth
- Demonstrated efficient resource utilization for multi-agent workflows

### Pattern Benefits
#### Performance Advantages
- Time Efficiency: Parallel execution dramatically reduces overall processing time compared to sequential analysis
- Resource Optimization: Multiple LLM calls executed concurrently rather than one-by-one
- Scalability: Pattern can easily accommodate additional specialized agents without increasing sequential processing time

### Analysis Quality
- Comprehensive Coverage: Multiple specialized perspectives ensure thorough content evaluation
- Domain Expertise: Each agent focuses on specific moderation aspects with targeted system prompts
- Consistent Structure: All agents return structured JSON responses for reliable aggregation